In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Qdrant
from qdrant_client.http.models import Distance, VectorParams
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser
import os

In [3]:
model_name = "CamiloGC93/bge-large-en-v1.5-etical"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
model_name=model_name,
model_kwargs=model_kwargs,
encode_kwargs=encode_kwargs
)   
embeddings = hf
file_paths = ["dataset/data1.pdf", "dataset/data2.pdf"]  # Add more files here

# List to store all loaded documents
documents = []

    # Loop through the file paths and load each PDF
for file_path in file_paths:
        loader = PyMuPDFLoader(file_path=file_path)
        docs = loader.load()  # Load documents from the current PDF
        documents.extend(docs)  # Add them to the overall documents list




text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 50)
rag_documents = text_splitter.split_documents(documents)

c:\Users\juanm\anaconda3\envs\llms\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
No sentence-transformers model found with name CamiloGC93/bge-large-en-v1.5-etical. Creating a new one with mean pooling.


In [4]:
quadran=os.environ["quadran_key"]
endpoint=os.environ["endpoint"]

In [11]:
import qdrant_client


client = qdrant_client.QdrantClient(
endpoint,
    api_key=quadran,
     https=True, 
     port=6333# For Qdrant Cloud, None for local instance
)


In [13]:
from qdrant_client.http.models import Distance, VectorParams
from qdrant_client import QdrantClient, models
client.create_collection(
    collection_name="EticalAI",
    vectors_config=models.VectorParams(size=512, distance=models.Distance.COSINE),
    optimizers_config=models.OptimizersConfigDiff(
        indexing_threshold=0,
    ),
)

True

In [58]:
from qdrant_client.models import PointStruct

In [14]:
client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='EticalAI')])

In [15]:
client.get_collection(collection_name="EticalAI")

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=None, indexed_vectors_count=0, points_count=0, segments_count=2, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=512, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=0, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantizat

In [19]:
quadran=os.environ["quadran_key"]
endpoint=os.environ["endpoint"]
qdrant = QdrantVectorStore.from_documents(
    rag_documents,
    embeddings,
    url=endpoint,
    prefer_grpc=True,
    api_key=quadran,
    collection_name="EticalAI",
)

In [23]:
retriever = qdrant.as_retriever()